### Agents

##### Question 1 - Adaptive Writing Format

In [52]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.agents import create_openai_functions_agent
from langchain import hub
import re
import os 

In [53]:
question = str(input("Enter the reformatting instruction followed by the text you want to reformat : "))
user_desiredFormat = str(input("Please specify the format you want to change your text to: "))

In [54]:
llm = ChatOpenAI(openai_api_key=os.getenv('OPENAI_API_KEY_PERSONAL'), temperature=0)

class userInput(BaseModel):
    question: str = Field()
class desiredFormat(BaseModel):
    n: str = Field()


def remove_citations(text):
    """
    Use this tool when you want to remove the citations from any text.
    """
    cleaned_text = re.sub(r'\[[0-9]+\]', '', text)
    return cleaned_text

def remove_extra_spaces(input_string):
    return ' '.join(input_string.split())

def desired_output_fun(user_input, user_desiredFormat):
    """
    Useful when user asks to convert the text format or style to another.
    """
    sysTmp = "You are an AI assistant that helps the user to convert one style text to another."
    sysMsgPmtTmp = SystemMessagePromptTemplate.from_template(sysTmp)
    humTmp = "Convert the following text as per the instruction \n\n {user_desiredFormat} \n\n{user_input}"
    HumMsgPmtTmp = HumanMessagePromptTemplate.from_template(humTmp)
    chatMsgPmtTmp = ChatPromptTemplate.from_messages([sysMsgPmtTmp, HumMsgPmtTmp])
    desiredOutputPrompt = chatMsgPmtTmp.format_prompt(user_input=user_input, user_desiredFormat=user_desiredFormat) 
    desiredOutput = llm.invoke(desiredOutputPrompt)
    return desiredOutput

tools = [
    Tool(
        name="removeCitations",
        func=remove_citations,
        description="A tool that removes citations form the give text.",
        args_schema=desiredFormat,
    ),
    Tool(
        name="removeSpaces",
        func=remove_extra_spaces,
        description="A tool that removes extra spaces form the give text.",
        args_schema=desiredFormat,
    ),
    Tool(
        func=desired_output_fun,
        name="desiredOutput",
        description="Useful only for when you need to convert or reformat one form of text style to another.",
        args_schema=userInput,
    ),
]

In [55]:
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

for step in agent_executor.iter({"input": [question,user_desiredFormat]}):
    if output := step.get("intermediate_step"):
        action, value = output[0]
        if action.tool == "removeSpaces":
            print(f"Checking whether {value} has white spaces...")
            assert remove_extra_spaces(value)
        if action.tool == "removeCitations":
            print(f"Checking whether {value} has citations...")
            assert remove_citations(value)
        if action.tool == "desiredOutput":
            print(f"Converting the text to user desired format - {value}...")
            assert desired_output_fun(value,user_desiredFormat)
        _continue = input("Should the agent continue (Y/n)?:\n") or "Y"
        if _continue.lower() != "y":
            break




> Entering new AgentExecutor chain...

Invoking: `removeCitations` with `{'n': "In the early 16th century, northern India, then under mainly Muslim rulers, fell again to the superior mobility and firepower of a new generation of Central Asian warriors. The resulting Mughal Empire did not stamp out the local societies it came to rule. Instead, it balanced and pacified them through new administrative practices and diverse and inclusive ruling elites, leading to more systematic, centralised, and uniform rule. Eschewing tribal bonds and Islamic identity, especially under Akbar, the Mughals united their far-flung realms through loyalty, expressed through a Persianised culture, to an emperor who had near-divine status. The Mughal state's economic policies, deriving most revenues from agriculture and mandating that taxes be paid in the well-regulated silver currency, caused peasants and artisans to enter larger markets. The relative peace maintained by the empire during much of the 17th cen

TypeError: desired_output_fun() missing 1 required positional argument: 'user_desiredFormat'

In [57]:
print(value)

In the early 16th century, northern India, then under mainly Muslim rulers, fell again to the superior mobility and firepower of a new generation of Central Asian warriors. The resulting Mughal Empire did not stamp out the local societies it came to rule. Instead, it balanced and pacified them through new administrative practices and diverse and inclusive ruling elites, leading to more systematic, centralised, and uniform rule. Eschewing tribal bonds and Islamic identity, especially under Akbar, the Mughals united their far-flung realms through loyalty, expressed through a Persianised culture, to an emperor who had near-divine status. The Mughal state's economic policies, deriving most revenues from agriculture and mandating that taxes be paid in the well-regulated silver currency, caused peasants and artisans to enter larger markets. The relative peace maintained by the empire during much of the 17th century was a factor in India's economic expansion, resulting in greater patronage of

<<< End Of Code >>>